In [1]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceTB/smoltalk", "all")

PEFT config

In [2]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,  # Rank dimension - typically between 4-32
    lora_alpha=32,  # LoRA scaling factor - typically 2x rank
    lora_dropout=0.1,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

Trainer loop

In [3]:
from trl import SFTConfig, SFTTrainer
from transformers import AutoTokenizer

device = "cuda"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

# Configure the SFTTrainer

training_args = SFTConfig(
    output_dir="./sft_output",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
    packing=True,
    eval_packing=False,
)

trainer = SFTTrainer(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    args=training_args,
    peft_config=peft_config,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Tokenizing train dataset:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (21818 > 16384). Running this sequence through the model will result in indexing errors


Packing train dataset:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/54948 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/54948 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/54948 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/54948 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Training
trainer.train()

/media/gleb/Data/Repos/huggingface-main-nlp-tasks/.venv/lib/python3.12/site-packages/transformers/models/qwen2/modeling_qwen2.py:335: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  freqs = (inv_freq_expanded.float().to(x.device) @ position_ids_expanded.float()).transpose(1, 2)


Merge

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from peft import PeftModel

# 1. Load the base model
base_model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", torch_dtype=torch.float16, device_map="auto"
)

# 2. Load the PEFT model with adapter
peft_model = PeftModel.from_pretrained(
    base_model, "path/to/adapter", torch_dtype=torch.float16
)

# 3. Merge adapter weights with base model
merged_model = peft_model.merge_and_unload()